# Mini-Project: SVM & LR Classification
Group 1: Anish Patel, Michael Weatherford, Tom Pengilly, Quynh Chau
October 3rd, 2020

Classification Modeling Summary
Project Goals:  
Our goal is to perform price classification analysis on the South American Property listings for various types of properties that are for sale in South America. Support Vector Machines (SVM) and Logistic Regression (LR) models were used to classify properties into three price class categories:Low, Average, High.  In addition, optimization techinques using GridSearchCV for SMV and LR, and SGDClassifier for LR will be use to improve model performance. 

Data Preprocessing and Sample Size:
Our original data set contained 463,724 rows of data.  For SVM and some Lr models, we randomly sampled 31,000 rows to be used in our training and test data set for classification modeling. Training/testing for both SVM and LR models used 80:20 split rule. In addition, sampled data was also scaled to enhance processing time. Dummy variables were created for property types and country.  

Summary Performance Results:
    
    
Comparative Advantages of SVM & LR
1.  Processing/Training time
In general, given the same data size, LR and optimization methods were faster to process than SVM & SVM optimization. When GridSearchCV was used to find optimal parameters with SVM, the process took 106.5 minutes as compared to the 29 minutes for GridSearchCV with LR.  In addition, SGDClassifier with LR also took less training time than GridSearchCV with LR.  We did not use SGDClassifier with SVM because we needed to have support vectors.

2. Accuracy



Interpretation of Weights from LR Models


Support Vectors for Classification Tasks

In [2]:
# Import the required libraries
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import svm 
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report, confusion_matrix


In [3]:
#Load in South American Real Estate "For Sale" Listing Data Set and drop Unamed index columns to create data set 'df'
df1 = pd.read_csv('Imputed_Dataset.csv')
df1.drop(df1.columns[df1.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
df1.dropna()
df1=df1.reset_index(drop=True)
df1.info()
df1.dtypes
df1.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463724 entries, 0 to 463723
Data columns (total 20 columns):
lat                409210 non-null float64
lon                409210 non-null float64
rooms              463724 non-null float64
bedrooms           463724 non-null float64
bathrooms          463724 non-null float64
surface_total      463724 non-null float64
surface_covered    463724 non-null float64
price              463723 non-null float64
log_price          463724 non-null float64
id                 463723 non-null object
start_date         463723 non-null object
end_date           463723 non-null object
created_on         463723 non-null object
property_type      463723 non-null object
l1                 463723 non-null object
l2                 463723 non-null object
l3                 441216 non-null object
title              463723 non-null object
description        463717 non-null object
price_class        463723 non-null object
dtypes: float64(9), object(11)
memory us

,lat,lon,rooms,bedrooms,bathrooms,surface_total,surface_covered,price,log_price,id,start_date,end_date,created_on,property_type,l1,l2,l3,title,description,price_class
0,NaN,NaN,4.0,3.0,2.0,198.0,150.0,385000.0,12.860999,EWeY8jVhb8ielLeKtfibVw==,2019-09-15,2020-01-20,2019-09-15,Casa,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,"Venta. Casa en una planta, 4 amb. con galería ...",MAM.(2) Hermosa propiedad en Barrio San Franci...,High
1,NaN,NaN,4.0,3.0,2.0,198.0,150.0,385000.0,12.860999,IrEeG8ewIcfLVGSb14kH1w==,2019-09-15,2020-01-20,2019-09-15,Casa,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,Venta. Casa en una planta.Lote interno. Barrio...,MAM. Hermosa propiedad en Barrio San Francisco...,High
2,-27.371199,-55.898454,7.0,3.0,2.0,173.0,173.0,195000.0,12.180755,hPuiyAjuBI92uEQdFoY4Fw==,2019-09-15,1970-01-01,2019-09-15,Casa,Argentina,Misiones,Posadas,Casa - Posadas,Inmueble centrico ideal para Local comercial ...,Average
3,NaN,NaN,3.0,3.0,1.0,49.0,40.0,85000.0,11.350407,9vIWv494LqC6crAkfpLeDg==,2019-09-15,2020-03-03,2019-09-15,Casa,Argentina,Santa Fe,Rosario,Cabaña en la Isla sobre riacho Los Marinos - V...,Lote de 20 metros de frente por 130 metros de ...,Low
4,-32.951146,-60.571979,3.0,3.0,1.0,49.0,40.0,85000.0,11.350407,aydrtgoZ/frzXDcV20McuA==,2019-09-15,2020-03-03,2019-09-15,Casa,Argentina,Entre Ríos,Victoria,Cabaña en la Isla sobre riacho Los Marinos - V...,Lote de 20 metros de frente por 130 metros de ...,Low


In [4]:
# Check on property types: there are 12 unique property types, including missing values, which will be dropped from analysis.
df1.property_type.unique()

array(['Casa', 'Departamento', 'Local comercial', 'PH', 'Lote', 'Otro',
       'Garaje', 'Oficina', 'Depósito', 'Casa de campo', 'Finca',
       'Parqueadero', nan], dtype=object)

In [5]:
# Check on country types: there are 5 unique South American countries and missing vlaues, which will be excluded from analysis.
df1.l1.unique()

array(['Argentina', 'Uruguay', 'Colombia', 'Ecuador', 'Perú', nan],
      dtype=object)

In [6]:
# In dataset df, drop specific columns that is not needed in Linear Regression
# Create data set 'df1' without remaining attributes
df1 = df1.drop(['lat', 'lon', 'start_date','end_date', 'created_on','l2','l3', 'title','description','id','log_price'], axis=1)
df1.head(5)

,rooms,bedrooms,bathrooms,surface_total,surface_covered,price,property_type,l1,price_class
0,4.0,3.0,2.0,198.0,150.0,385000.0,Casa,Argentina,High
1,4.0,3.0,2.0,198.0,150.0,385000.0,Casa,Argentina,High
2,7.0,3.0,2.0,173.0,173.0,195000.0,Casa,Argentina,Average
3,3.0,3.0,1.0,49.0,40.0,85000.0,Casa,Argentina,Low
4,3.0,3.0,1.0,49.0,40.0,85000.0,Casa,Argentina,Low


In [7]:
#Defining variable data types process data for SVM modeling and omit rows with missing data 
string_vars = ['price_class','property_type','l1']
df1[string_vars] = df1[string_vars].astype(str)
df1=df1.dropna()

In [8]:
# Get Dummies Variables process property type values for SVM modeling
dummies = pd.get_dummies(df1.property_type)
dummies

,Casa,Casa de campo,Departamento,Depósito,Finca,Garaje,Local comercial,Lote,Oficina,Otro,PH,Parqueadero
0,1,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
463718,0,0,1,0,0,0,0,0,0,0,0,0
463719,0,0,1,0,0,0,0,0,0,0,0,0
463720,0,0,0,0,0,0,0,0,0,1,0,0
463721,0,0,1,0,0,0,0,0,0,0,0,0


In [10]:
# Join dummies dataset with original dataset to create a combined data set, 'mergedf1'
mergedf1 = pd.concat([df1,dummies], axis = 'columns')
mergedf1 = mergedf1.drop(['property_type'], axis = 1)
mergedf1

,rooms,bedrooms,bathrooms,surface_total,surface_covered,price,l1,price_class,Casa,Casa de campo,Departamento,Depósito,Finca,Garaje,Local comercial,Lote,Oficina,Otro,PH,Parqueadero
0,4.0,3.0,2.0,198.0,150.0,385000.0,Argentina,High,1,0,0,0,0,0,0,0,0,0,0,0
1,4.0,3.0,2.0,198.0,150.0,385000.0,Argentina,High,1,0,0,0,0,0,0,0,0,0,0,0
2,7.0,3.0,2.0,173.0,173.0,195000.0,Argentina,Average,1,0,0,0,0,0,0,0,0,0,0,0
3,3.0,3.0,1.0,49.0,40.0,85000.0,Argentina,Low,1,0,0,0,0,0,0,0,0,0,0,0
4,3.0,3.0,1.0,49.0,40.0,85000.0,Argentina,Low,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463718,5.0,2.0,1.0,214.0,107.0,129000.0,Uruguay,Average,0,0,1,0,0,0,0,0,0,0,0,0
463719,2.0,2.0,1.0,63.0,62.0,210000.0,Uruguay,High,0,0,1,0,0,0,0,0,0,0,0,0
463720,4.0,2.0,2.0,255.0,128.0,150000.0,Uruguay,Low,0,0,0,0,0,0,0,0,0,1,0,0
463721,4.0,3.0,2.0,114.0,114.0,98500.0,Uruguay,Average,0,0,1,0,0,0,0,0,0,0,0,0


In [11]:
# Create dummy variable for country values and process data for SVM modeling
dummies2 = pd.get_dummies(mergedf1.l1)
dummies2

,Argentina,Colombia,Ecuador,Perú,Uruguay
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0
...,...,...,...,...,...
463718,0,0,0,0,1
463719,0,0,0,0,1
463720,0,0,0,0,1
463721,0,0,0,0,1


In [12]:
# Join dummies dataset with original dataset, delete unnecessary variables l1 and price to create data set 'mergedf2'
mergedf2 = pd.concat([mergedf1,dummies2], axis = 'columns')
mergedf2 = mergedf2.drop(['l1'], axis = 1)
mergedf2

,rooms,bedrooms,bathrooms,surface_total,surface_covered,price,price_class,Casa,Casa de campo,Departamento,...,Lote,Oficina,Otro,PH,Parqueadero,Argentina,Colombia,Ecuador,Perú,Uruguay
0,4.0,3.0,2.0,198.0,150.0,385000.0,High,1,0,0,...,0,0,0,0,0,1,0,0,0,0
1,4.0,3.0,2.0,198.0,150.0,385000.0,High,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,7.0,3.0,2.0,173.0,173.0,195000.0,Average,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,3.0,3.0,1.0,49.0,40.0,85000.0,Low,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,3.0,3.0,1.0,49.0,40.0,85000.0,Low,1,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463718,5.0,2.0,1.0,214.0,107.0,129000.0,Average,0,0,1,...,0,0,0,0,0,0,0,0,0,1
463719,2.0,2.0,1.0,63.0,62.0,210000.0,High,0,0,1,...,0,0,0,0,0,0,0,0,0,1
463720,4.0,2.0,2.0,255.0,128.0,150000.0,Low,0,0,0,...,0,0,1,0,0,0,0,0,0,1
463721,4.0,3.0,2.0,114.0,114.0,98500.0,Average,0,0,1,...,0,0,0,0,0,0,0,0,0,1


In [13]:
# Create random sample of 31,000 rows from original data set into a new data set, mergedf3, used for classification modeling
mergedf3 = mergedf2.sample(n=31000,random_state=6)

In [14]:
# Assign arrays test SVM models using data set mergedf2
X= mergedf3.drop(['price_class'], axis = 1)
X=np.asarray(X)
y=np.asarray(mergedf3['price_class'])
X.shape

(31000, 23)

In [15]:
#Split data into 80:20 train and test data sets, and normalize the data for faster SVM model processing time
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state=6)
X_train = preprocessing.scale(X_train)
X_test = preprocessing.scale(X_test)

In [17]:
# Build SVM 'linear' model 
from sklearn.svm import SVC
classifier0 = svm.SVC(kernel = 'linear', gamma = 1, C=1)
classifier0.fit(X_train,y_train)
y_predict0 = classifier0.predict(X_test)
print (classification_report(y_test,y_predict0))

              precision    recall  f1-score   support

     Average       0.51      0.81      0.63      2094
        High       0.83      0.95      0.89      2067
         Low       1.00      0.23      0.38      2039

    accuracy                           0.67      6200
   macro avg       0.78      0.67      0.63      6200
weighted avg       0.78      0.67      0.63      6200



In [16]:
# Build SVM 'rbf' model to classify price_class (Low, Average, High) using attributes: total rooms, bedrooms, bathrooms, surface total,
# surface covered, price and property types.
from sklearn.svm import SVC
classifier1 = svm.SVC(kernel ='rbf', gamma = 1, C=1)
classifier1.fit(X_train,y_train)
y_predict1=classifier1.predict(X_test)
print (classification_report(y_test, y_predict1))

              precision    recall  f1-score   support

     Average       0.52      0.79      0.63      2094
        High       0.81      0.92      0.86      2067
         Low       0.89      0.30      0.44      2039

    accuracy                           0.67      6200
   macro avg       0.74      0.67      0.64      6200
weighted avg       0.74      0.67      0.64      6200



In [17]:
# Use GridSearch to find optimal parameters for SVM model
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
param_grid = [{'C':(0.5,1,10,100),'gamma':['scale',1,0.1,0.01, 0.001, 0.0001],'kernel':['rbf']}]
optimal_params = GridSearchCV(SVC(), param_grid, cv=10, scoring = 'accuracy', verbose = 2)
optimal_params.fit(X_train,y_train)


Fitting 10 folds for each of 24 candidates, totalling 240 fits
[CV] C=0.5, gamma=scale, kernel=rbf ..................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................... C=0.5, gamma=scale, kernel=rbf, total=  27.0s
[CV] C=0.5, gamma=scale, kernel=rbf ..................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   26.9s remaining:    0.0s


[CV] ................... C=0.5, gamma=scale, kernel=rbf, total=  29.6s
[CV] C=0.5, gamma=scale, kernel=rbf ..................................
[CV] ................... C=0.5, gamma=scale, kernel=rbf, total=  30.2s
[CV] C=0.5, gamma=scale, kernel=rbf ..................................
[CV] ................... C=0.5, gamma=scale, kernel=rbf, total=  27.1s
[CV] C=0.5, gamma=scale, kernel=rbf ..................................
[CV] ................... C=0.5, gamma=scale, kernel=rbf, total=  27.9s
[CV] C=0.5, gamma=scale, kernel=rbf ..................................
[CV] ................... C=0.5, gamma=scale, kernel=rbf, total=  27.6s
[CV] C=0.5, gamma=scale, kernel=rbf ..................................
[CV] ................... C=0.5, gamma=scale, kernel=rbf, total=  27.0s
[CV] C=0.5, gamma=scale, kernel=rbf ..................................
[CV] ................... C=0.5, gamma=scale, kernel=rbf, total=  27.3s
[CV] C=0.5, gamma=scale, kernel=rbf ..................................
[CV] .

[Parallel(n_jobs=1)]: Done 240 out of 240 | elapsed: 106.5min finished


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid=[{'C': (0.5, 1, 10, 100),
                          'gamma': ['scale', 1, 0.1, 0.01, 0.001, 0.0001],
                          'kernel': ['rbf']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=2)

In [18]:
print(optimal_params.best_params_)

{'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}


In [25]:
# Build SVM 'rbf' model to classify price_class (Low, Average, High) using attributes: total rooms, bedrooms, bathrooms, surface total,
# surface covered, price, and property types using optimal parameters found in GridSearchCV using kernel = rbf
from sklearn.svm import SVC
classifier2 = svm.SVC(kernel ='rbf', gamma = 0.01, C=100)
classifier2.fit(X_train,y_train)
y_predict2=classifier2.predict(X_test)
print (classification_report(y_test, y_predict2))
print (classifier2.score(X_test, y_test))

              precision    recall  f1-score   support

     Average       0.50      0.82      0.62      2094
        High       0.84      0.94      0.89      2067
         Low       0.98      0.23      0.37      2039

    accuracy                           0.67      6200
   macro avg       0.78      0.66      0.63      6200
weighted avg       0.77      0.67      0.63      6200

0.665483870967742


In [23]:
# Build Logistic Regression Model 
LR = LogisticRegression(multi_class= 'auto',solver = 'sag', max_iter = 4000)
LR_fit = LR.fit(X_train,y_train)
y_predLR = LR.predict(X_test)
print(LR_fit)
print(classification_report(y_test,y_predLR))
print(LR.score(X_test,y_test))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=4000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)
              precision    recall  f1-score   support

     Average       0.52      0.84      0.64      2094
        High       0.85      0.95      0.90      2067
         Low       1.00      0.24      0.39      2039

    accuracy                           0.68      6200
   macro avg       0.79      0.68      0.64      6200
weighted avg       0.79      0.68      0.64      6200

0.6790322580645162


In [27]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
LR1=LogisticRegression(multi_class='auto', solver = 'sag', max_iter=5000)
param_grid = {'penalty': ['l2'], 'C': [0.5,1,10,100]}
LR2 = GridSearchCV(LR1,param_grid, cv = 10, verbose = 2)
LR2.fit(X_train,y_train)
print(LR2.best_params_)


Fitting 10 folds for each of 4 candidates, totalling 40 fits
[CV] C=0.5, penalty=l2 ...............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................................ C=0.5, penalty=l2, total=  21.8s
[CV] C=0.5, penalty=l2 ...............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   21.7s remaining:    0.0s


[CV] ................................ C=0.5, penalty=l2, total=  21.5s
[CV] C=0.5, penalty=l2 ...............................................
[CV] ................................ C=0.5, penalty=l2, total=  21.3s
[CV] C=0.5, penalty=l2 ...............................................
[CV] ................................ C=0.5, penalty=l2, total=  22.2s
[CV] C=0.5, penalty=l2 ...............................................
[CV] ................................ C=0.5, penalty=l2, total=  21.5s
[CV] C=0.5, penalty=l2 ...............................................
[CV] ................................ C=0.5, penalty=l2, total=  21.7s
[CV] C=0.5, penalty=l2 ...............................................
[CV] ................................ C=0.5, penalty=l2, total=  21.6s
[CV] C=0.5, penalty=l2 ...............................................
[CV] ................................ C=0.5, penalty=l2, total=  21.4s
[CV] C=0.5, penalty=l2 ...............................................
[CV] .

[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed: 29.0min finished


{'C': 100, 'penalty': 'l2'}


In [31]:
# Build Optimal Logistic Regression Model Based on GridSearchCV
LR3 = LogisticRegression(multi_class= 'auto',class_weight ='balanced', C=100, penalty ='l2',solver = 'sag', max_iter=4000)
LR_fit3 = LR3.fit(X_train,y_train)
y_predLR3 = LR3.predict(X_test)
print(LR_fit3)
print(classification_report(y_test,y_predLR3))
print(LR.score(X_test,y_test))

LogisticRegression(C=100, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=4000, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)
              precision    recall  f1-score   support

     Average       0.51      0.81      0.62      2094
        High       0.83      0.95      0.89      2067
         Low       1.00      0.23      0.38      2039

    accuracy                           0.67      6200
   macro avg       0.78      0.66      0.63      6200
weighted avg       0.78      0.67      0.63      6200

0.6790322580645162


In [36]:
LR3.intercept_
print(LR3.coef_)

[[-1.22069970e-02  1.42789014e-02 -6.47087310e-02 -6.98596249e-04
  -1.91226355e-02  5.86259879e+00  3.05398092e-01 -6.24275439e-02
  -8.27419695e-01  3.98364915e-01  6.48497094e-02 -2.28493646e-01
   4.79606589e-01  6.47184732e-01  2.75255927e-02  8.12713738e-01
  -3.73541166e-01  9.91732933e-02  3.05554567e-03 -3.16089774e-03
  -2.46529617e-03 -7.09889133e-03  1.29342503e-02]
 [ 3.05337663e-02  1.32122696e-01  5.88359711e-02 -1.32579375e-03
   5.63815166e-06  5.90764390e+01 -1.88819282e+00 -3.37505997e-03
   3.21239845e+00 -1.98980153e+00 -1.16764770e+00  1.44488407e+00
  -1.03311375e+00 -6.61487010e-01  3.48534076e-02 -2.67594026e+00
   9.59261205e-01 -1.15277326e+00  4.54478265e-02 -8.60910784e-02
  -2.42169254e-03  5.15169852e-02  3.06776380e-02]
 [-1.83267693e-02 -1.46401597e-01  5.87275987e-03  2.02439000e-03
   1.91169974e-02 -6.49390378e+01  1.58279473e+00  6.58026039e-02
  -2.38497875e+00  1.59143661e+00  1.10279799e+00 -1.21639043e+00
   5.53507159e-01  1.43022779e-02 -6.237

In [37]:
confusion_matrix(y_test,y_predLR3)

array([[1689,  405,    0],
       [  97, 1970,    0],
       [1555,    7,  477]], dtype=int64)

In [55]:
#Logistic Regression Optimization Using Stochastic Gradient Descent
LR4 = SGDClassifier(loss = 'log', penalty = 'l2', class_weight = 'balanced')
LR4.fit(X_train, y_train)
y_predLR4 = LR4.predict(X_test)
print(classification_report(y_test,y_predLR4))
print(LR4.score(X_test,y_test))

              precision    recall  f1-score   support

     Average       0.51      0.69      0.58      2094
        High       0.75      0.97      0.85      2067
         Low       1.00      0.34      0.51      2039

    accuracy                           0.67      6200
   macro avg       0.75      0.66      0.64      6200
weighted avg       0.75      0.67      0.64      6200

0.6659677419354839


In [56]:
print(LR4.coef_)

[[ 9.54769767e-03  2.76993757e-02 -3.07269280e-02 -1.27542591e-01
   1.16422088e-01 -1.65732285e+00  1.62763863e-01 -9.57910980e-02
  -1.28354101e-01 -7.34310504e-02 -5.51602551e-02  8.36445399e-02
   1.35103131e-01  2.80123772e-03 -1.99874261e-03 -1.46205217e-02
  -8.15298077e-02 -1.10988357e-01 -5.42019144e-03 -3.04069502e-02
  -2.82159489e-01  3.99235338e-02  1.05565292e-01]
 [ 4.61969108e-02  5.12736892e-02  2.32940311e-02 -1.44061526e-01
  -9.81736171e-02  2.70924566e+01 -1.14014198e+00 -7.48539577e-02
   2.05381882e+00 -1.08638938e+00 -5.68285686e-01  7.68544675e-01
  -7.09901626e-01 -6.96615787e-01  1.74145218e-03 -1.80076837e+00
   7.11512051e-01 -6.58992414e-01  1.62089123e-01 -1.82657240e-01
  -3.72790580e-01  6.01028588e-02 -6.48203373e-03]
 [-3.47698057e-02 -6.56970142e-02 -8.57053349e-03 -1.79557130e-02
   3.24692878e-01 -3.30187431e+01  6.67097823e-01 -1.73472175e-01
  -7.66316580e-01  6.37762098e-01  4.48513247e-01 -5.67465390e-01
   3.08089360e-02 -2.49534625e-01  6.591